In [1]:
import os
import numpy as np
import pandas as pd
from utils import get_unfindicators
pd.options.io.excel.xlsx.writer = 'xlsxwriter'

target = r"C:\datas\企业绩效数据\非财务绩效数据\deals"
# 对数据进行简单的预处理，将一些非财务报表中的excel文件转化为txt,方便后面操作
path = r"C:\datas\企业绩效数据\非财务绩效数据"
indicators = get_unfindicators()[0]

In [2]:
# 先处理高管人数、持股及薪酬情况===>董事人数;独立董事人数
file_ = r'高管人数、持股及薪酬情况表160101119(仅供盐城工学院使用)\\CG_ManagerShareSalary.txt'
dataset = pd.read_csv(os.path.join(path, file_), sep='\t', header=0, dtype={'Symbol': 'object'}, parse_dates=['Enddate'])
dataset.rename(columns={'Enddate': 'EndDate'}, inplace=True)
names = {item for item in dataset.columns.values.tolist() if item in indicators}
dataset = dataset[(dataset.StatisticalCaliber == 2)][list(names)]
dataset = dataset[(dataset['EndDate'].dt.month==12)&(dataset['EndDate'].dt.year>2006)][list(names)]
dataset['EndDate'] = dataset['EndDate'].dt.year
dataset.sort_values(by=['Symbol', 'EndDate'],inplace=True)
dataset.to_csv(os.path.join(target, file_.split('\\')[-1]), sep='\t', index=False)

In [3]:
# 十大股东股权集中文件====>股权集中指标4(%)
file_ = r'十大股东股权集中文件163516241(仅供盐城工学院使用)\\HLD_CR.txt'
dataset = pd.read_csv(os.path.join(path, file_), sep='\t', header=0, dtype={'Stkcd': 'object'}, parse_dates=['Reptdt'])
dataset.rename(columns={'Reptdt': 'EndDate', 'Stkcd': 'Symbol'}, inplace=True)
names = {item for item in dataset.columns.values.tolist() if item in indicators}
dataset = dataset[(dataset['EndDate'].dt.month==12)&(dataset['EndDate'].dt.year>2006)][list(names)]
dataset['EndDate'] = dataset['EndDate'].dt.year
dataset.sort_values(by=['Symbol', 'EndDate'],inplace=True)

dataset.to_csv(os.path.join(target, file_.split('\\')[-1]), sep='\t', index=False)

In [4]:
# 研发人员数量;研发人员数量占比;研发投入金额；研发投入占营业收入比例(%)
file_ = r'研发投入情况表161958173(仅供盐城工学院使用)\\PT_LCRDSPENDING.txt'
dataset = pd.read_csv(os.path.join(path, file_), sep='\t', header=0, dtype={'Symbol': 'object'}, parse_dates=['EndDate'])
names = {item for item in dataset.columns.values.tolist() if item in indicators}
dataset = dataset[(dataset['Source'] == 0)&(dataset['StateTypeCode'] == 1)&(dataset['EndDate'].dt.year>2006)&(dataset['EndDate'].dt.month==12)][list(names)]
dataset['EndDate'] = dataset['EndDate'].dt.year
dataset.to_csv(os.path.join(target, file_.split('\\')[-1]), sep='\t', index=False)

In [5]:
# 前五大客户、供应商情况表、本期销售额占年度销售总额比例(%)；本期采购额占年度采购总额比例(%)
file_ = r'前五大客户、供应商情况表160624235(仅供盐城工学院使用)\\FN_FN010.txt'
dataset = pd.read_csv(os.path.join(path, file_), sep='\t', header=0, dtype={'stkcd': 'object'}, parse_dates=['accper'])
names = {item for item in dataset.columns.values.tolist() if item in indicators}
dataset = dataset[(dataset.DataSources==0)&(dataset.typrep == 1) & (dataset.FN_Fn01001 == 6)&(dataset.accper.dt.month == 12)&(dataset.accper.dt.year >2006)][list(names)]
salse = dataset[dataset['FN_Fn01004'] == 1][['stkcd', 'accper','FN_Fn01006']]
salse.columns = ['Symbol', 'EndDate', '本期销售额占年度销售总额比例(%)']
perchase = dataset[dataset['FN_Fn01004'] == 2][['stkcd', 'accper', 'FN_Fn01006']]
perchase.columns = ['Symbol', 'EndDate','本期采购额占年度采购总额比例(%)']
dataset = pd.merge(salse, perchase, on=['Symbol', 'EndDate', ], how='outer')
dataset['EndDate'] = dataset['EndDate'].dt.year
dataset.sort_values(by=['Symbol', 'EndDate'],inplace=True)

dataset.to_csv(os.path.join(target, file_.split('\\')[-1]), sep='\t', index=False)

In [6]:
# 现金流量表2007-2022===>应付职工薪酬
file_ = r'现金流量表(直接法)163422942(仅供盐城工学院使用)\\FS_Comscfd.txt'
dataset = pd.read_csv(os.path.join(path, file_), sep='\t', header=0, dtype={'Stkcd': 'object'}, parse_dates=['Accper'])
dataset.rename(columns={'Accper': 'EndDate', 'Stkcd': 'Symbol'}, inplace=True)
names = {item for item in dataset.columns.values.tolist() if item in indicators}
dataset = dataset[(dataset.Typrep == 'A')&(dataset.EndDate.dt.year >2016)&(dataset.EndDate.dt.month ==12)][list(names)]
dataset['EndDate'] = dataset['EndDate'].dt.year
dataset.sort_values(by=['Symbol', 'EndDate'],inplace=True)

dataset.to_csv(os.path.join(target, file_.split('\\')[-1]), sep='\t', index=False)

In [7]:
# 社会责任报告基本信息:纳税总额，捐献总额
file_ = r'上市公司社会责任报告基本信息表163211165(仅供盐城工学院使用)\\CSRR_BasicInfo.txt'
dataset = pd.read_csv(os.path.join(path, file_), sep='\t', header=0, dtype={'Symbol': 'object'}, parse_dates=['EndDate'])
names = {item for item in dataset.columns.values.tolist() if item in indicators}
dataset = dataset[(dataset['EndDate'].dt.month==12)&(dataset['EndDate'].dt.year>2006)][list(names)]
dataset['EndDate'] = dataset['EndDate'].dt.year
dataset.sort_values(by=['Symbol', 'EndDate'],inplace=True)
dataset.to_csv(os.path.join(target, file_.split('\\')[-1]), sep='\t', index=False)

In [8]:
# 治理综合信息文件160356990===>员工人数;董事长与总经理兼任情况;股东总数
file_ = r'治理综合信息文件160356990(仅供盐城工学院使用)\\CG_Ybasic.txt'
dataset = pd.read_csv(os.path.join(path, file_), sep='\t', header=0, dtype={'Stkcd': 'object'}, parse_dates=['Reptdt'])
dataset.rename(columns={'Reptdt': 'EndDate', 'Stkcd': 'Symbol'}, inplace=True)
names = {item for item in dataset.columns.values.tolist() if item in indicators}
dataset = dataset[(dataset['EndDate'].dt.month ==12)&(dataset['EndDate'].dt.year>2006)][list(names)]
dataset['EndDate'] = dataset['EndDate'].dt.year
dataset.sort_values(by=['Symbol', 'EndDate'],inplace=True)

dataset.to_csv(os.path.join(target, file_.split('\\')[-1]), sep='\t', index=False)

In [4]:
# 主营业务收入
income_path = r'C:\datas\企业绩效数据\非财务绩效数据\主营业务收入'
file_ = 'FN_Fn048.txt'
files = []
for dir_ in os.listdir(income_path):
    path_ = os.path.join(income_path, dir_)
    for entry in os.scandir(path_):
        if not entry.name.endswith('[txt].txt'):
            files.append(os.path.join(path_, entry.name))
dataset = pd.DataFrame(columns=['Symbol','EndDate','Fn04804'])

for fpath in files:
    dataset_ = pd.read_csv(fpath, sep='\t', header=0, dtype={'Stkcd': 'object'}, parse_dates=['Accper'])
    dataset_.rename(columns={'Accper': 'EndDate', 'Stkcd': 'Symbol'}, inplace=True)
    names = {item for item in dataset.columns.values.tolist() if item in indicators}
    dataset_ = dataset_[(dataset_['Typrep']==1)&(dataset_['Sgnyea']==1)&(dataset_['fn04814']==1)&(dataset_['EndDate'].dt.year>2006)&(dataset_['Fn04802']=='合计')&(dataset_['Fn04801']==1)][list(names)]
    dataset_['EndDate'] = dataset_['EndDate'].dt.year
    dataset = pd.concat([dataset,dataset_])
dataset.drop_duplicates(subset=['Symbol','EndDate'],inplace=True)
dataset.sort_values(by=['Symbol', 'EndDate'],inplace=True)
dataset.reset_index(inplace=True)
dataset.drop('index',axis=1,inplace=True)
dataset.to_csv(os.path.join(target, file_), sep='\t', index=False)

C:\Users\shelhen\AppData\Local\Temp\ipykernel_8412\4106467154.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataset = pd.concat([dataset,dataset_])


In [10]:
file_ = r'组织韧性1990-2022\\内部控制指数及评级.csv'
dataset = pd.read_csv(os.path.join(path, file_), encoding='gbk',parse_dates=['报告期'])

dataset['Symbol'] = dataset['证券代码'].apply(lambda x: x[:-3])
dataset['EndDate'] = pd.to_datetime(dataset['报告期'])
dataset['EndDate'] = dataset['EndDate'].dt.year
dataset = dataset[dataset['EndDate']>2006][['Symbol', 'EndDate', '内部控制指数评分']]

dataset['内部控制指数评分'].replace(to_replace=0, value=np.NaN, inplace=True)
dataset.rename(columns={'内部控制指数评分': '内部控制指数'}, inplace=True)
dataset.sort_values(by=['Symbol', 'EndDate'],inplace=True)

dataset.to_csv(os.path.join(target, file_.split('\\')[-1].split('.')[0] + '.txt'), sep='\t', index=False)

C:\Users\shelhen\AppData\Local\Temp\ipykernel_25488\1053635027.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset['内部控制指数评分'].replace(to_replace=0, value=np.NaN, inplace=True)


In [11]:
file_ = r'组织韧性1990-2022\\组织韧性.dta'

dataset = pd.io.stata.read_stata(os.path.join(path, file_))
dataset['Symbol'] = dataset['id'].apply(lambda x: '{:0>6d}'.format(x))
dataset['EndDate'] = pd.to_datetime(dataset['year'].apply(lambda x: f'{x}-12-31'))
dataset = dataset[dataset['EndDate'].dt.year > 2006][['Symbol', 'EndDate', 'sd', 'growth', 'Score']]
dataset.columns = ['Symbol', 'EndDate', '财务波动', '长期绩效增长', '组织韧性']
dataset['EndDate'] = dataset['EndDate'].dt.year
dataset.sort_values(['Symbol', 'EndDate'], inplace=True, ascending=True)
dataset.to_csv(os.path.join(target, file_.split('\\')[-1].split('.')[0] + '.txt'), sep='\t', index=False)

In [12]:
# 专利
file_ = r'国内外专利申请获得情况表161114761(仅供盐城工学院使用)\\PT_LCDOMFORAPPLY.txt'
dataset = pd.read_csv(os.path.join(path, file_), sep='\t', header=0, dtype={'Symbol': 'object'}, parse_dates=['EndDate'])
dataset['EndDate'] =dataset['EndDate'].dt.year
names = {item for item in dataset.columns.values.tolist() if item in indicators}
dataset = dataset[(dataset['EndDate']>2017)&(dataset['StateTypeCode']==1)&(dataset['Area']==1)&(dataset['ApplyTypeCode']=='S5203')][['Symbol', 'EndDate','Patents']]
dataset.dropna(inplace=True)
file_ = r'专利授权情况表(1992-2017)142534725(仅供盐城工学院使用)\\PT_GRANT.txt'
dataset_ = pd.read_csv(os.path.join(path, file_), sep='\t', header=0, dtype={'Symbol': 'object'}, parse_dates=['EndDate'])
dataset_['EndDate'] =dataset_['EndDate'].dt.year
dataset_= dataset_[(dataset_['EndDate']>2006)&(dataset_['ClassifySign']==4)][['Symbol','EndDate','Grants']]
dataset_.columns=['Symbol','EndDate','Patents']
dataset = pd.concat([dataset, dataset_])
dataset.sort_values(['Symbol', 'EndDate'], inplace=True, ascending=True)
dataset.to_csv(os.path.join(target, file_.split('\\')[-1]), sep='\t', index=False)

In [13]:
# 专利2
_path2 = r'专利授权情况表(1992-2017)142534725(仅供盐城工学院使用)\\上市公司专利申请和授权数据.xlsx'
pd.options.io.excel.xlsx.writer = 'xlsxwriter'

dataset = pd.read_excel(os.path.join(path, _path2),index_col=None,header=0,dtype={'股票代码':'object'})
dataset.rename(columns={'股票代码':'Symbol','year':'EndDate'},inplace=True)
dataset = dataset[(dataset['公司类型']=='上市公司本身')&(dataset['EndDate']>2006)]
patents = dataset.columns.tolist()[3:]
dataset['Patents2'] = dataset[patents].apply(lambda x:x.sum(), axis=1)
dataset = dataset[['Symbol','EndDate','Patents2']]
# print(dataset.duplicated(subset=['Symbol', 'EndDate']).sum())
dataset.to_csv(os.path.join(target, _path2.split('\\')[-1].split('.')[0]+'.txt'), sep='\t', index=False)

In [14]:
# 上市公司控制人文件:
file_ = r'上市公司控制人文件005917664(仅供盐城工学院使用)\\HLD_Contrshr.csv'
dataset = pd.read_csv(os.path.join(path, file_), sep=',', header=0, dtype={'Stkcd': 'object'}, parse_dates=['Reptdt'])
dataset.rename(columns={'Reptdt': 'EndDate', 'Stkcd': 'Symbol'}, inplace=True)
names = {item for item in dataset.columns.values.tolist() if item in indicators}
# 
dataset = dataset[(dataset['S0701a']==1)&(dataset['EndDate'].dt.year>2006)&(dataset['EndDate'].dt.year<2023)][list(names)]
dataset.drop_duplicates(subset=['Symbol', 'EndDate'],keep='first',inplace=True)
dataset['EndDate'] = dataset['EndDate'].dt.year
# dataset.sort_values(by=['Symbol', 'EndDate'],inplace=True)
def judge_(x):
    set_1 = {'1100', '2000', '2100', '2120'}
    if type(x) == str:
        set_2 = set(x.split(','))
        return 1 if (set_1 & set_2) else 0
    else:
        return 0
dataset['股权性质'] = dataset['S0702b'].apply(lambda x: judge_(x))
dataset.drop('S0702b',axis=1,inplace=True)
dataset_ = pd.read_excel(r'C:\datas\年报文本\csmar数字化转型数据\数字化转型指数\CSMAR企业数字化转型指数统计表（年）-合作数据\EDT_DigTransIndexStaY.xlsx',index_col=None,skiprows=[0,2],header=0,dtype={'证券代码':'object','统计年度':'int'})
dataset_ = dataset_[['证券代码','统计年度','股权性质编码']]
dataset_.columns=['Symbol', 'EndDate','S0702b']
dataset_['S0702b'] = dataset_['S0702b'].apply(lambda x:1  if ('1' in str(x)) else 0)
dataset = pd.merge(dataset_,dataset,on=['Symbol', 'EndDate'],how='outer')
dataset.drop_duplicates(subset=['Symbol', 'EndDate'],keep='first',inplace=True)
dataset['S0702b'] = dataset.groupby('Symbol')['S0702b'].fillna(method='ffill')
dataset['S0702b'] = dataset.groupby('Symbol')['S0702b'].fillna(dataset['股权性质'])
dataset = dataset[['Symbol', 'EndDate','S0702b','Seperation']]
dataset.sort_values(by=['Symbol', 'EndDate'],inplace=True)
dataset.reset_index(inplace=True)
dataset.drop('index',axis=1,inplace=True)
dataset.to_csv(os.path.join(target, file_.split('\\')[-1].split('.')[0]+'.txt'), sep='\t', index=False)

C:\envs\digital\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\shelhen\AppData\Local\Temp\ipykernel_25488\2842916684.py:26: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use Series.fillna instead
  dataset['S0702b'] = dataset.groupby('Symbol')['S0702b'].fillna(method='ffill')
C:\Users\shelhen\AppData\Local\Temp\ipykernel_25488\2842916684.py:26: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dataset['S0702b'] = dataset.groupby('Symbol')['S0702b'].fillna(method='ffill')
C:\Users\shelhen\AppData\Local\Temp\ipykernel_25488\2842916684.py:27: FutureWarning: SeriesGroupBy.fillna is deprecated a

In [ ]:
# print(dataset.duplicated(subset=['Symbol', 'EndDate']).sum())
# dataset[dataset.duplicated(subset=['Symbol', 'EndDate'])]